### PtH2 system size calculation
Following script is used to calculate Electrolyzer size for first and second scenario per each industrial sector. Storage size is also calculated for 3rd scenario which then be used as input storage size value for capacity optimizatin script.     

In [4]:
import pandas as pd
#data input
industrialDemandH2 = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/input/2019/industrial_demand.csv')
electrolyzers = pd.read_csv('/Users/kanankhasmammadov/Desktop/Thesis - Electrolyzer market participation/flexABLE_w_electrolyzer/input/2019/electrolyzers.csv')
industrialDemandH2['Timestamp'] = pd.date_range(start=f'1/1/2016', end=f'12/31/2016 23:45', freq='15T')            

display(industrialDemandH2)
display(electrolyzers)

,Unnamed: 0,Iron_steel,Timestamp
0,0,48084.11,2016-01-01 00:00:00
1,1,48034.88,2016-01-01 00:15:00
2,2,48104.89,2016-01-01 00:30:00
3,3,48076.33,2016-01-01 00:45:00
4,4,48420.37,2016-01-01 01:00:00
...,...,...,...
35131,35131,48110.13,2016-12-31 22:45:00
35132,35132,47964.95,2016-12-31 23:00:00
35133,35133,48008.64,2016-12-31 23:15:00
35134,35134,48125.15,2016-12-31 23:30:00


,name,technology,minLoad,maxLoad,installedCapacity,effElec,minDowntime,coldStartUpCost,maxAllowedColdStartups,standbyCons,comprCons,maxSOC,company
0,Iron_steel,PEM,0.05,1.2,11850,0.63,0.5,50,3000,0.0005,0.0012,200000,Iron_steel


### Electrolyzer capacity calculation

The electrolyzer capacity should be able to produce hydrogen at each step without having need for storage. Therefore the maximum hydrogen demand value is extracted on study year and respective electrolyzer power is calculated for it. Since data frequency is 15 minute, that should also be considered in calculation.

In [5]:
energyContentH2_LHV = 0.03333 #MWh/kg or lower heating value of H2
efficiency = electrolyzers['effElec']
maxLoad = electrolyzers['maxLoad']
# Calculate electrolyzer capacity
iron_steel_maxDemand = max(industrialDemandH2['Iron_steel'])

iron_steel_Capacity = round((iron_steel_maxDemand * energyContentH2_LHV / efficiency[0]/0.25/maxLoad[0]), -1)
# Metal_Capacity = round((Metal_maxDemand * energyContentH2_LHV / efficiency[1] / 0.25), -1)

print(iron_steel_Capacity, 'MW iron_steel_Capacity')


11830.0 MW iron_steel_Capacity


### Storage capacity calculation
Storage capacity is calculated based on assumption of having 1 day of enough hydrogen storage to proceed with operations.

In [9]:
from statistics import mean
# Calculate the maxSOC for H2 storage
demandSum_iron_steel = [] #weekly or daily demand sum

industrialDemandH2['Date'] = industrialDemandH2['Timestamp'].dt.date
unique_days = industrialDemandH2['Date'].unique()
for day in unique_days:
    dailyIntervalDemand = industrialDemandH2[industrialDemandH2['Date'] == day]
    daily_sum_iron_steel = sum(dailyIntervalDemand['Iron_steel'])
    
    demandSum_iron_steel.append(daily_sum_iron_steel) 

storageCap_iron_steel = round(mean(demandSum_iron_steel), -2)

print(storageCap_iron_steel/2, 'storageCap_iron_steel')


2763950.0 storageCap_iron_steel
